## What I want to achieve

Based on my current knowledge of python and machine learning, I want to be able to run an ensemble to algos to predict titanic survival. Gosh, we're still into that. I hope I dont ever have to touch this or the iris dataset in a years time...Thanks to [Bugra Akyildiz](http://bugra.github.io/work/notes/2014-11-22/an-introduction-to-supervised-learning-scikit-learn/) for a great blog on supervised learning. Check it out and weep with joy!

Ok, so what do we need to do? - Binary classification of survival

## Admin stuff

In [16]:
%matplotlib inline 

from IPython.display import Image

import pandas as pd
pd.set_option('display.max_columns', None) # Display any number of columns

import numpy as np
from matplotlib import pyplot
%matplotlib inline

# Set seaborn aesthetic parameters to defaults
import seaborn as sns
sns.set()

## Read the training dataset

In [2]:
train = pd.read_csv('train.csv')
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Data wrangling from the lessons learned during the exploratory session

### Quick explorations

In [3]:
# check the class distribution of the response variable
train.Survived.value_counts()/train.shape[0]*100

0    61.616162
1    38.383838
Name: Survived, dtype: float64

Almost 60:40 split. Looks cool to me. No class imbalance.

In [4]:
# quick test to see if passenger id is indeed unique and can be used as the index
print len(train.PassengerId), len(train.PassengerId.unique())

891 891


In [5]:
# fill up age variable with righteous values i.e. median age
print train.Age.isnull().sum()
train['Age'].fillna(train['Age'].median(), inplace = True)
print train.Age.isnull().sum()

177
0


In [6]:
# check out SibSp and Parch features
print train.SibSp.isnull().sum()
print train.Parch.isnull().sum()
print train.Fare.isnull().sum()
print train.Embarked.isnull().sum()

0
0
0
2


In [7]:
train.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [8]:
train.Embarked.value_counts()

# most of them seem to be from Southampton S, so encode the 2 remaining null values as 'S'

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [9]:
# check how many rows have cabin level information
print train.Cabin.isnull().sum(), "pax dont have cabin level information"

# what about ticket info
print train.Ticket.isnull().sum(), "pax are travelling ticketless. However,...."

# however, multiple people could be travelling on the same ticket. Lets see if thats the case
print len(train.Ticket) - len(train.Ticket.drop_duplicates()), "pax are travelling on another passengers tickets."

687 pax dont have cabin level information
0 pax are travelling ticketless. However,....
210 pax are travelling on another passengers tickets.


### Create a function to do the data cleansing for us

In [10]:
def titanic_wrangling(raw_dataset):
    
    # convert the passengerid column into index. I've already checked that this field is unique per row.
    clean_dataset = raw_dataset.set_index('PassengerId', drop=True)
    
    # there's heaps of null ages. I got this from the exploratory session in the previous workbooks. Fill with 
    # median age of the passengers
    clean_dataset['Age'].fillna(clean_dataset['Age'].median(), inplace = True)
    
    # theres 2 null embarked rows. I'm coding these as 'S' fo reasons described above
    clean_dataset['Embarked'].fillna('S', inplace = True)
  
    # Encode string values into discrete integers
    #import & instantiate
    from sklearn import preprocessing
    label_encoder = preprocessing.LabelEncoder()
    # fit & transform SEX variable into discrete integer values
    clean_dataset['Sex'] = label_encoder.fit_transform(clean_dataset['Sex'])
    # fit & transform EMBARKED variable into discrete integer values
    clean_dataset['Embarked'] = label_encoder.fit_transform(clean_dataset['Embarked'])
    # fit & transform TICKET variable into discrete integer values. Given 210 pax have dup tickets, this could be 
    # predictive.
    clean_dataset['Ticket'] = label_encoder.fit_transform(clean_dataset['Ticket'])
    
    # Get the response variable as a numpy matrix
    label = clean_dataset['Survived'].as_matrix().astype(int)
    
    # Drop redundant columns from the clean dataset
    # note that I'm removing the cabin information for now, because there is too much scope for overfitting here 
    # notice also that I'm dropping Survived because I've already created the response variable
    clean_dataset.drop(['Name','Cabin','Survived'], axis=1, inplace=True)
    
    features = clean_dataset.as_matrix().astype(np.float)
    
    return features, label

__Future Improvement #1:__ Note here that encoding categorical features into integers has introduced implicit ordering in the Sex and Ticket variable. Strings are not inherently ordered, but integers are. We could transform these string features by creating dummy features. This can be achieved by: <br\> a) using sklearn.preprocessing.OneHotEncoder, or <br\> b) using the pandas method pd.get_dummies()<br\>
__Future Improvement #2:__ Dont impute age variable only using the median age. We can use some more smarts on this. Based on the gender, class and the salutation (Mr, Mrs, Master, Miss etc.), we can get a better sense of the ages by grouping them on these lines<br\>
__Future Improvement #3:__ Apply some more smarts around the family variables SibSp and Parch instead of simply putting it there. Example, children with parents surely have a much better chance of survival. So create variables such as married with spouses present, children with parents etc. kind of variables under the head family status

### Use the data wrangling function to give us our features and labels

In [11]:
X, y = titanic_wrangling(train)
print X.shape, y.shape

(891, 8) (891,)


In [12]:
X

array([[   3.    ,    1.    ,   22.    , ...,  523.    ,    7.25  ,    2.    ],
       [   1.    ,    0.    ,   38.    , ...,  596.    ,   71.2833,    0.    ],
       [   3.    ,    0.    ,   26.    , ...,  669.    ,    7.925 ,    2.    ],
       ..., 
       [   3.    ,    0.    ,   28.    , ...,  675.    ,   23.45  ,    2.    ],
       [   1.    ,    1.    ,   26.    , ...,    8.    ,   30.    ,    0.    ],
       [   3.    ,    1.    ,   32.    , ...,  466.    ,    7.75  ,    1.    ]])

Looks like the features are clearly in need of some scaling and standardisation...

### Perform scaling

In [14]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
# the standardscaler subtracts mean and divides by stdev

# standardised X - this has mean 0 and stdev 1
X

array([[ 0.82737724,  0.73769513, -0.56573646, ...,  0.91896631,
        -0.50244517,  0.58595414],
       [-1.56610693, -1.35557354,  0.66386103, ...,  1.28262456,
         0.78684529, -1.9423032 ],
       [ 0.82737724, -1.35557354, -0.25833709, ...,  1.64628282,
        -0.48885426,  0.58595414],
       ..., 
       [ 0.82737724, -1.35557354, -0.1046374 , ...,  1.67617254,
        -0.17626324,  0.58595414],
       [-1.56610693,  0.73769513, -0.25833709, ..., -1.64656796,
        -0.04438104, -1.9423032 ],
       [ 0.82737724,  0.73769513,  0.20276197, ...,  0.63501397,
        -0.49237783, -0.67817453]])

### Cross validation

In [24]:
cv_url = 'https://www.researchgate.net/profile/Min-Yuan_Cheng/publication/260399256/figure/fig4/ \
AS:297311155572744@1447895911905/Fig-4-Differential-Evolution-based-cross-validation-process.png'
Image(url=cv_url)

# Perform cross validation because ¯\_(ツ)_/¯